## Recommendations with MovieTweetings: Collaborative Filtering

**Collaborative filtering** is one of the most popular methods for making recommendations  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, I will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook I will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.

The dataset are two: reviews.csv and movies.csv. The reviews.csv dataset is a dataset of reviews given to movies with columns as user_id, movie_id, and ratings; while movies.csv has the movie name, the movie_id, the genre, dates, and categories the movie belongs to. The goal of this analysis will be to build a recommendation system that can prescribe movies to any user using information from the neighbors of that particular user. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(reviews.head())

   user_id  movie_id  rating   timestamp                 date  month_1  \
0        1     68646      10  1381620027  2013-10-12 23:20:27        0   
1        1    113277      10  1379466669  2013-09-18 01:11:09        0   
2        2    422720       8  1412178746  2014-10-01 15:52:26        0   
3        2    454876       8  1394818630  2014-03-14 17:37:10        0   
4        2    790636       7  1389963947  2014-01-17 13:05:47        0   

   month_2  month_3  month_4  month_5    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  

### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  If some measures are on a 5 point scale, while others are on a 100 point scale, you are likely to have non-optimal results due to the difference in variability of your features.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](images/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [2]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items.head()

,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


In [3]:
#user_items[user_items['movie_id'] == 1454468]

In [4]:
#user_items[user_items['movie_id'] == 5495]

In [8]:
user_items.movie_id.max()

8439854

### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [3]:
# Create user-by-item matrix
my_df = user_items.pivot(index = 'user_id', columns = 'movie_id')['rating']

In [4]:
my_df.head(20)

NameError: name 'my_df' is not defined

In [4]:
user_by_movie = my_df

In [5]:
user_by_movie.head()

movie_id,8,10,12,25,91,417,439,443,628,833,...,8144778,8144868,8206708,8289196,8324578,8335880,8342748,8342946,8402090,8439854
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
gy = user_by_movie[~user_by_movie[12].isnull()].values[0]

In [53]:
gn = []
gh = [gn.append(i) for i in gy if i != np.nan]

In [58]:
gy = gy[~np.isnan(gy)]

In [59]:
gy

array([ 10.,   7.,   9.,   6.,   8.,   4.,   8.,   8.,   7.,   9.,   8.,
         6.,   7.,   5.,   7.,   6.,   5.,   7.,   6.,   8.,   7.,   6.,
         6.,   8.,   7.,   7.,   7.,   7.,   5.,   6.,   6.,   7.,   6.,
         9.,   9.,   6.,   6.,   6.,   8.])

In [15]:
user_by_movie.shape

(53968, 31245)

In [36]:
#user_by_movie[8]

Check your results below to make sure your matrix is ready for the upcoming sections.

In [6]:
assert movies.shape[0] == user_by_movie.shape[1], "Oh no! Your matrix should have {} columns, and yours has {}!".format(movies.shape[0], user_by_movie.shape[1])
assert reviews.user_id.nunique() == user_by_movie.shape[0], "Oh no! Your matrix should have {} rows, and yours has {}!".format(reviews.user_id.nunique(), user_by_movie.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

Looks like you are all set! Proceed!


`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [9]:
#np.isnan(user_by_movie.iloc[1,8])

In [16]:
i_watched1 = []
i_watched2 = [i_watched1.append(i) for i, j in user_by_movie.loc[2,:].items() if not np.isnan(j)]

In [12]:
#user_by_movie.iloc[1]
i_watched1 = []
i_watched2 = [i_watched1.append(i) for i in user_by_movie.iloc[1] if not np.isnan(i)]

In [17]:
i_watched1

[422720,
 454876,
 790636,
 816711,
 1091191,
 1103275,
 1322269,
 1390411,
 1398426,
 1431045,
 1433811,
 1454468,
 1535109,
 1675434,
 1798709,
 2017038,
 2024544,
 2294629,
 2361509,
 2381249,
 2726560,
 2883512,
 3079380]

In [7]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    i_watched1 = []
    i_watched2 = [i_watched1.append(i) for i, j in user_by_movie.loc[user_id,:].items() if not np.isnan(j)]
    movies = i_watched1
    return movies


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''
    movies_seen = {}
    for k in range(1,user_by_movie.shape[0]):
        movies_seen[k] = movies_watched(k)
    return movies_seen


# Use your function to return dictionary
movies_seen = create_user_movie_dict()

In [11]:
#movies_seen

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [8]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''
    
    # Do things to create updated dictionary
    movies_to_analyze = {}
    for i,j in movies_seen.items():
        if len(j) > lower_bound:
            movies_to_analyze[i] = j
        else:
            continue
    
    return movies_to_analyze


# Use your function to return your updated dictionary
movies_to_analyze = create_movies_to_analyze(movies_seen)

In [9]:
movies_to_analyze

{2: [422720,
  454876,
  790636,
  816711,
  1091191,
  1103275,
  1322269,
  1390411,
  1398426,
  1431045,
  1433811,
  1454468,
  1535109,
  1675434,
  1798709,
  2017038,
  2024544,
  2294629,
  2361509,
  2381249,
  2726560,
  2883512,
  3079380],
 3: [1790864, 2170439, 2203939],
 7: [1764234, 1790885, 2053463],
 8: [385002, 1220198, 1462900, 1512685, 1631707, 1986994, 1999995],
 9: [65207, 363163, 985699],
 17: [68646, 71562, 99674],
 22: [359950,
  770828,
  831387,
  1091191,
  1170358,
  1210819,
  1430132,
  1490017,
  1535108,
  1663662,
  1843866,
  1872181,
  2103281,
  2179136,
  2209764,
  2267998,
  2333784,
  2334879,
  2713180,
  2872732],
 24: [816692, 2245084, 2267998, 2488496, 2802144, 2820852, 3300542],
 25: [88847,
  327056,
  482571,
  790724,
  795421,
  1130884,
  1231583,
  1232829,
  1647668,
  2119532,
  2294449,
  2404463,
  3230660,
  3774114],
 26: [3954660, 5222918, 7291268],
 30: [111161, 454921, 1454468, 2024469, 2231253, 2304771],
 31: [106977,
  434

In [10]:
# Run the tests below to check that your movies_to_analyze matches the solution
assert len(movies_to_analyze) == 23512, "Oops!  It doesn't look like your dictionary has the right number of individuals."
assert len(movies_to_analyze[2]) == 23, "Oops!  User 2 didn't match the number of movies we thought they would have."
assert len(movies_to_analyze[7])  == 3, "Oops!  User 7 didn't match the number of movies we thought they would have."
print("If this is all you see, you are good to go!")

If this is all you see, you are good to go!


### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users.  Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [34]:
#movies_to_analyze

In [54]:
Sim_movie = [mov for mov in movies_to_analyze[2] if mov in movies_to_analyze[104]]

In [55]:
Sim_movie

[454876, 816711, 1454468, 1535109]

In [56]:
user_by_movie.loc[2, Sim_movie[:]]

movie_id
454876     8.0
816711     8.0
1454468    8.0
1535109    8.0
Name: 2, dtype: float64

In [57]:
user_by_movie.loc[104, Sim_movie[:]]

movie_id
454876     9.0
816711     7.0
1454468    7.0
1535109    9.0
Name: 104, dtype: float64

In [13]:
compute_correlation(2,104)

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


nan

In [11]:
from scipy.stats import pearsonr

In [12]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''
    corr = []
    try:
        sim_movie = [mov for mov in movies_to_analyze[user1] if mov in movies_to_analyze[user2]]
        corr = pearsonr(user_by_movie.loc[user1, sim_movie[:]], user_by_movie.loc[user2, sim_movie[:]])[0]
    except KeyError:
        pass

    return corr #return the correlation

In [13]:
# Test your function against the solution
assert compute_correlation(2,2) == 1.0, "Oops!  The correlation between a user and itself should be 1.0."
assert round(compute_correlation(2,66), 2) == 0.76, "Oops!  The correlation between user 2 and 66 should be about 0.76."
assert np.isnan(compute_correlation(2,104)), "Oops!  The correlation between user 2 and 104 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

If this is all you see, then it looks like your function passed all of our tests!


/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [50]:
sim_movie = [mov for mov in movies_to_analyze[2] if mov in movies_to_analyze[2]]
corr = pearsonr(user_by_movie.loc[2, sim_movie[:]], user_by_movie.loc[2, sim_movie[:]])[0]

In [51]:
corr

1.0

### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 2 and 104 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data - 2,526,710 of them in fact. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.




In [ ]:
# Which movies did both user 2 and user 104 see?
#Standard deviation of similar rating is zero, and zero cannot be used as a denominator

In [ ]:
# What were the ratings for each user for those movies?


`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [14]:
from scipy.spatial import distance
def compute_euclidean_dist(user1, user2):
    
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    
    eucl = []
    try:
        sim_movie = [mov for mov in movies_to_analyze[user1] if mov in movies_to_analyze[user2]]
        eucl = distance.euclidean((user_by_movie.loc[user1, sim_movie[:]]), (user_by_movie.loc[user2, sim_movie[:]]))
    except KeyError:
        pass
    return eucl#return the euclidean distance

In [15]:
# Read in solution euclidean distances
import pickle
df_dists = pd.read_pickle("data/Term2/recommendations/lesson1/data/dists.p")

In [16]:
# Test your function against the solution
assert compute_euclidean_dist(2,2) == df_dists.query("user1 == 2 and user2 == 2")['eucl_dist'][0], "Oops!  The distance between a user and itself should be 0.0."
assert round(compute_euclidean_dist(2,66), 2) == round(df_dists.query("user1 == 2 and user2 == 66")['eucl_dist'][1], 2), "Oops!  The distance between user 2 and 66 should be about 2.24."
assert np.isnan(compute_euclidean_dist(2,104)) == np.isnan(df_dists.query("user1 == 2 and user2 == 104")['eucl_dist'][4]), "Oops!  The distance between user 2 and 104 should be 2."

print("If this is all you see, then it looks like your function passed all of our tests!")

/opt/conda/lib/python3.6/site-packages/numexpr/cpuinfo.py:42: UserWarning: [Errno 12] Cannot allocate memory
  warnings.warn(str(e), UserWarning, stacklevel=stacklevel)


If this is all you see, then it looks like your function passed all of our tests!


In [28]:
df_dists.head()

,user1,user2,eucl_dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000


In [21]:
df_dists.shape

(31813482, 3)

In [22]:
user_by_movie.shape

(53968, 31245)

In [25]:
(5396*5396)+5396

29122212

### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [17]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    user_df = df_dists[df_dists['user1'] == user].sort_values(['eucl_dist'], ascending = True,axis = 0)
    closest_neighbors = list(user_df['user2'])
    
    return closest_neighbors
#find_closest_neighbors(2)

In [32]:
user_items.head()

,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


In [18]:
neighbors = find_closest_neighbors(2)
neighbors_ = user_items[user_items['user_id'].isin(neighbors)]

In [21]:
nt = neighbors_[neighbors_['rating']>=7].sort_values(['rating'], ascending = False)
#nt

In [22]:
nt

,user_id,movie_id,rating
346733,26292,318462,10
330334,24933,1619029,10
330247,24933,99685,10
330252,24933,108052,10
682330,51885,137523,10
330253,24933,111161,10
330254,24933,111341,10
93594,7140,5442430,10
330256,24933,113277,10
682334,51885,167260,10


In [33]:
#user1_movies = user_items[user_items['rating']> 7 & user_items['user_id'] == 2]['movie_id'].values

In [36]:
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
movies[movies['movie_id'].isin([2,])]['movie'].values

In [ ]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    user_df = df_dists[df_dists['user1'] == user].sort_values(['eucl_dist'], ascending = True,axis = 0)
    closest_neighbors = list(user_df['user2'])
    
    return closest_neighbors
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    movies_liked = user_items[user_items['rating']>= min_rating].sort_values(['rating'], ascending = False)
    movies_liked = movies_liked[movies_liked['user_id'] == user_id]['movie_id'].values
    #neighbors = find_closest_neighbors(user_id)
    #n#eighbors_ = user_items[user_items['user_id'].isin(neighbors)]
    #movies_liked = neighbors_[neighbors_['rating']>7].sort_values(['rating'], ascending = False)
  
    return movies_liked


def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = movies[movies['movie_id'].isin(movie_ids)]['movie'].values
    
    return list(movie_lst)
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    neighb = find_closest_neighbors(user)
    liked = []
    for i in neighb:
        liked.extend(movies_liked(i, min_rating=7))
    if len(liked) == 0:
        recommendations = []
    elif len(liked) < 11:
        recommendations = liked
    else:
        recommendations = np.random.choice(liked, 10, replace = False)
    return recommendations

def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    

    # Make the recommendations for each user
    user_list = list(user_items['user_id'].unique())
    all_recs = {}
    for k in user_list:
        all_recs[k] = make_recommendations(k, num_recs=10)
        
    
    return all_recs

all_recs = all_recommendations(10)

In [46]:
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    movies_liked = user_items[user_items['rating']> min_rating].sort_values(['rating'], ascending = False)
    movies_liked = movies_liked[movies_liked['user_id'] == user_id]['movie_id'].values
    #neighbors = find_closest_neighbors(user_id)
    #n#eighbors_ = user_items[user_items['user_id'].isin(neighbors)]
    #movies_liked = neighbors_[neighbors_['rating']>7].sort_values(['rating'], ascending = False)
  
    return movies_liked

In [48]:
neighb = find_closest_neighbors(2)
liked = []
for i in neighb:
    liked.extend(movies_liked(i, min_rating=7))
recommendations = np.random.choice(liked, 10, replace = False)

In [49]:
recommendations

array([2802144, 2024544, 1596363,   78788,  451279, 2035630, 2832470,
       1754656, 5013056,  770828])

In [41]:
user_list = list(user_items['user_id'].unique())

In [43]:
len(user_list)

53968

In [19]:
# This loads our solution dictionary so you can compare results
all_recs_sol = pd.read_pickle("data/Term2/recommendations/lesson1/data/all_recs.p")

In [ ]:
assert all_recs[2] == make_recommendations(2), "Oops!  Your recommendations for user 2 didn't match ours."
assert all_recs[26] == make_recommendations(26), "Oops!  It actually wasn't possible to make any recommendations for user 26."
assert all_recs[1503] == make_recommendations(1503), "Oops! Looks like your solution for user 1503 didn't match ours."
print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [ ]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': # letter here,
    'The two methods used to estimate user similarity were: ': # letter here,
    'There was an issue with using the correlation coefficient.  What was it?': # letter here
}

t.test_recs(sol_dict)

Additionally, let's take a closer look at some of the results.  There are two solution files that you read in to check your results, and you created these objects

* **df_dists** - a dataframe of user1, user2, euclidean distance between the two users
* **all_recs_sol** - a dictionary of all recommendations (key = user, value = list of recommendations)  

`9.` Use these two objects along with the cells below to correctly fill in the dictionary below and complete this notebook!

In [ ]:
a = 567
b = 1503
c = 1319
d = 1325
e = 2526710
f = 0
g = 'Use another method to make recommendations - content based, knowledge based, or model based collaborative filtering'

sol_dict2 = {
    'For how many pairs of users were we not able to obtain a measure of similarity using correlation?': # letter here,
    'For how many pairs of users were we not able to obtain a measure of similarity using euclidean distance?': # letter here,
    'For how many users were we unable to make any recommendations for using collaborative filtering?': # letter here,
    'For how many users were we unable to make 10 recommendations for using collaborative filtering?': # letter here,
    'What might be a way for us to get 10 recommendations for every user?': # letter here   
}

t.test_recs2(sol_dict2)

In [ ]:
# Use the cells below for any work you need to do!

In [ ]:
# Users without recs


In [ ]:
# NaN euclidean distance values


In [ ]:
# Users with less than 10 recs
